In [ ]:
import base
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rushd as rd
import scipy as sp
import seaborn as sns

# enables concurrent editing of base.py
from importlib import reload
reload(base)

### Load data

In [ ]:
base_path = rd.datadir/'instruments'/'data'/'attune'
data, quantiles, stats, metadata = base.load_data(base_path, rd.datadir/'projects'/'miR-iFFL'/'plasmids', 'tuning', 'designs')

In [ ]:
# Create dicts to specify colors/markers
metadata_dict = metadata.set_index('construct').to_dict('dict')
main_palette = metadata_dict['color']
main_markers = metadata_dict['markers']

ts_label = {'na': 'base', 'NT': 'OL', 'T': 'CL', 'none': '–'}

### Set up figure

In [ ]:
base_size = base.font_sizes['base_size']
smaller_size = base.font_sizes['smaller_size']

sns.set_style('ticks')
sns.set_context('paper', font_scale=1.0, rc={'font.size': base_size, 'font.family': 'sans-serif', 'font.sans-serif':['Arial']})
plt.rcParams.update({'axes.titlesize': base_size, 'axes.labelsize': base_size, 'xtick.labelsize': smaller_size, 'ytick.labelsize': smaller_size,
                     'pdf.fonttype': 42, 
                     'ytick.major.size': 3, 'xtick.major.size': 3, 'ytick.minor.size': 2, 'ytick.major.pad': 2, 'xtick.major.pad': 2, 
                     'lines.linewidth': 1,
                     'axes.spines.right': False, 'axes.spines.top': False, 'axes.labelpad': 2})

In [ ]:
# Create the overall figure, gridspec, and add subfigure labels
fig = plt.figure(figsize=(6.8504,4))
fig_gridspec = matplotlib.gridspec.GridSpec(2, 3, figure=fig,
    wspace=0.4, hspace=0.4, height_ratios=[2.125,(4-2.125)], width_ratios=[1,1,1])
subfigures = {
    'A': fig.add_subfigure(fig_gridspec[0,0]),
    'B': fig.add_subfigure(fig_gridspec[0,1]),
    'C': fig.add_subfigure(fig_gridspec[0,2]),
    'D': fig.add_subfigure(fig_gridspec[1,0]),
    'E': fig.add_subfigure(fig_gridspec[1,1]),
    'F': fig.add_subfigure(fig_gridspec[1,2]),
}
for label, subfig in subfigures.items():
    subfig.add_artist(matplotlib.text.Text(x=0, y=1, text=f'{label}', fontsize=base.font_sizes['subpanel_label'], 
                                           fontweight='bold', verticalalignment='top',transform=subfig.transSubfigure))

output_path = rd.rootdir/'output'/'fig_overview'/'fig_overview.pdf'
fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['E']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.5, bottom=0.4, top=0.35, right=0.15)
axes = subfig.subplots(1,2, gridspec_kw=dict(width_ratios=(1,0.3)))

biorep = 2
plot_df = quantiles[(((quantiles['miR']=='miR.FF5') & (quantiles['group']=='controller') & (quantiles['design']==1) & (quantiles['ts'].isin(['FF4x1','FF5x1','FF6x1']))) | (quantiles['group']=='base')) &
                         (quantiles['promoter']=='EF1a') & (quantiles['biorep']==biorep) & (quantiles['exp']!='exp90.4')]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=main_palette, 
             legend=False, dashes=False, style='construct', markers=main_markers, ax=ax, markersize=4, markeredgewidth=0.5,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlabel='marker', title='Initial circuit design', ylim=(2e1,1e5))
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(quantiles.loc[(quantiles['group']=='marker') & (quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=smaller_size)

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.despine(ax=ax, bottom=True)
ax.set(xlabel='', ylim=axes[0].get_ylim(), ylabel='', yticklabels=[])
ax.get_xaxis().set_visible(False)

for ax in axes: ax.minorticks_off()

fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['F']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.35, bottom=0.4, top=0.35, right=0.05)
axes = subfig.subplots(1,3, gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.7))

# stat gmean
plot_df = stats[(((stats['miR']=='miR.FF5') & (stats['group']=='controller') & (stats['design']==1) & (stats['ts'].isin(['FF4x1','FF5x1','FF6x1']))) | (stats['group']=='base')) &
                         (stats['promoter']=='EF1a')]
xlim = (-0.5, len(plot_df['ts_kind'].unique())-0.5)
scatter_kwargs = dict(s=4, jitter=0.2, linewidth=0.5, edgecolor='white')

ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts_kind', y='output_gmean', hue='construct', palette=main_palette,
                  legend=False, ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Mean', xlim=xlim, xlabel='', ylabel='', yscale='log', ylim=(1e3,2e4),)

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts_kind', y='output_std', hue='construct', palette=main_palette,
                  legend=False, ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Std.', xlim=xlim, xlabel='', ylabel='', yscale='log', ylim=(1e3,2.5e4))

# # stat std
# ax = axes[0]
# for construct, group in plot_df.groupby('construct'):
#     sns.stripplot(data=group, x='ts_kind', y='output_std', hue='construct', palette=main_palette,
#                   legend=False, ax=ax, marker=main_markers[construct], **scatter_kwargs)
# ax.set(title='Std.', xlim=xlim, xlabel='', ylabel='', yscale='log', ylim=(1e3,2.5e4))

# # stat CV
# ax = axes[1]
# for construct, group in plot_df.groupby('construct'):
#     sns.stripplot(data=group, x='ts_kind', y='output_variation', hue='construct', palette=main_palette,
#                   legend=False, ax=ax, marker=main_markers[construct], **scatter_kwargs)
# ax.set(title='CV', xlim=xlim, xlabel='', ylabel='', ylim=(0.9,1.3))

# slope
ax = axes[2]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts_kind', y='slope', hue='construct', palette=main_palette,
                  legend=False, ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Slope', xlim=xlim, xlabel='', ylabel='', ylim=(0.4,1.25))

for ax in axes:
    ax.set_xticklabels([ts_label[x.get_text()] for x in ax.get_xticklabels()], rotation=45, ha='right',)
    sns.despine(ax=ax)

fig.savefig(rd.outfile(output_path))

In [ ]:
# Save to OneDrive
fig.savefig(rd.outfile(rd.datadir/'manuscripts'/'2024_miR-iFFL'/'figures'/'links'/'fig_overview.pdf'))